In [1]:
import pandas as pd
import numpy as np

# read csv file
df = pd.read_csv("behaviour.csv")
print(df)

         id  symbol     rate       wacc  transaction  quantity  net_quantity  \
0       518    AHPC    93.00    93.3441          NaN      1000           NaN   
1       519   RADHI   169.25   169.9270          NaN       123           NaN   
2       520   GGBSL   774.40   777.2653          NaN       100           NaN   
3       521    ILBS   721.17   723.8383          NaN       241           NaN   
4       522    MERO   621.22   623.5185          NaN       115           NaN   
...     ...     ...      ...        ...          ...       ...           ...   
2904  12728   NESDO  2082.10  2089.8038          NaN       200           NaN   
2905  12727   HIDCL   198.05   198.7828          NaN      1210           NaN   
2906   9812    NICA   703.00   705.6011          NaN       500           NaN   
2907   9813  SANIMA   237.00   237.8769          NaN       500           NaN   
2908  23455     NTC   897.00   900.3189          NaN       100           NaN   

         amount                        

In [2]:
unique_values = df[['symbol', 'user_id']].drop_duplicates()
print(unique_values.count())

symbol     451
user_id    451
dtype: int64


## Average Days Between Each Trade /Average Holding Period

In [3]:
# according to user_id and symbol 
def average_days_btn_trade(df):
    # Convert the 'transaction_date' column to datetime
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    # Filter the DataFrame to keep only the SELL transactions
    sell_df = df[df['transaction_type'] == 'SELL']
    # Calculate the difference in days between the most recent BUY and the SELL transactions
    result = []
    for (user_id, symbol), symbol_sell_df in sell_df.groupby(['user_id', 'symbol']):
        for index, sell_row in symbol_sell_df.iterrows():
            buy_rows = df[(df['transaction_type'] == 'BUY') & (df['user_id'] == user_id) & (df['symbol'] == symbol) & (df['transaction_date'] < sell_row['transaction_date'])]
            if not buy_rows.empty:
                most_recent_buy = buy_rows.iloc[-1]
                days = (sell_row['transaction_date'] - most_recent_buy['transaction_date']).days
                result.append({'user_id': user_id, 'symbol': symbol, 'Buy_Date': most_recent_buy['transaction_date'], 'Sell_Date': sell_row['transaction_date'], 'Days': days})
    result_df = pd.DataFrame(result)
    # Calculate the average days grouped by User_ID and Symbol
    average_days_by_user_symbol = result_df.groupby(['user_id', 'symbol'])['Days'].mean().reset_index()
    return average_days_by_user_symbol
average_days_by_user_symbol = average_days_btn_trade(df)
print(average_days_by_user_symbol)

     user_id  symbol        Days
0       2056  ACLBSL   13.714286
1       2056    ADLB    3.000000
2       2056    AHPC  344.250000
3       2056     AIL    3.000000
4       2056   AKJCL   20.000000
..       ...     ...         ...
356    33387    SSHL    9.500000
357    33387     STC   11.000000
358    33387    SWMF    4.000000
359    33387     TRH   11.750000
360    33387   UNHPL    6.000000

[361 rows x 3 columns]


## Average Profit in Each Trade

In [4]:
# according to user_id and symbol 
def calculate_average_profit(df):
    result = []
    # Calculate metrics for each user-stock combination
    for (user, stock), group in df.groupby(['user_id', 'symbol']):
        avg_profit_in_each_trade = group['profit_loss_percentage'].mean()
        result.append({'user_id': user, 'symbol': stock, 'avg_profit_in_each_trade': avg_profit_in_each_trade})
    result_df = pd.DataFrame(result)
    return result_df
average_profit = calculate_average_profit(df)
print(average_profit)

     user_id  symbol  avg_profit_in_each_trade
0       2056  ACLBSL                   -0.0400
1       2056    ADLB                    0.1800
2       2056    AHPC                    0.0975
3       2056     AIL                    0.0600
4       2056   AKBSL                       NaN
..       ...     ...                       ...
446    33387    SSHL                   -0.0700
447    33387     STC                   -0.0250
448    33387    SWMF                   -0.0200
449    33387     TRH                   -0.0350
450    33387   UNHPL                   -0.0600

[451 rows x 3 columns]


## Average Cummulative Profit

In [5]:
def calculate_average_cumulative_profit(df):
    grouped_data = df.groupby(['user_id', 'symbol'])
    # Calculate metrics for each user-stock combination
    result = []
    # Calculate metrics for each user-stock combination
    for (user, stock), group in grouped_data:
        average_cumulative_profit = group['profit_loss_percentage'].cumsum().mean()
        result.append({'user_id': user, 'symbol': stock, 'average_cumulative_profit': average_cumulative_profit})
    result_df = pd.DataFrame(result)
    return result_df
average_cumulative_profit = calculate_average_cumulative_profit(df)
print(average_cumulative_profit)


     user_id  symbol  average_cumulative_profit
0       2056  ACLBSL                  -0.128571
1       2056    ADLB                   0.180000
2       2056    AHPC                   0.112500
3       2056     AIL                   0.060000
4       2056   AKBSL                        NaN
..       ...     ...                        ...
446    33387    SSHL                  -0.162500
447    33387     STC                  -0.020000
448    33387    SWMF                  -0.020000
449    33387     TRH                  -0.135000
450    33387   UNHPL                  -0.060000

[451 rows x 3 columns]


## Average Success Rate

In [6]:
def calculate_average_success_rate(df):
    grouped_data = df.groupby(['user_id', 'symbol'])
    # Calculate metrics for each user-stock combination
    result = []
    # Calculate metrics for each user-stock combination
    for (user, stock), group in grouped_data:
        successful_trades = group[group['profit_loss_percentage'] > 0]
        sucess_rate = (len(successful_trades) / len(group)) * 100
        
        result.append({'user_id': user, 'symbol': stock, 'sucess_rate': sucess_rate})
    result_df = pd.DataFrame(result)
    return result_df
average_success_rate = calculate_average_success_rate(df)
print(average_success_rate)

     user_id  symbol  sucess_rate
0       2056  ACLBSL     7.142857
1       2056    ADLB    50.000000
2       2056    AHPC    42.857143
3       2056     AIL    50.000000
4       2056   AKBSL     0.000000
..       ...     ...          ...
446    33387    SSHL     0.000000
447    33387     STC    25.000000
448    33387    SWMF     0.000000
449    33387     TRH    12.500000
450    33387   UNHPL     0.000000

[451 rows x 3 columns]


## Sharpe Ratio

In [7]:
# Create a function to calculate Sharpe ratio for a given group
def calculate_sharpe_ratio(df):
    grouped_data = df.groupby(['user_id', 'symbol'])
    result = []
    for (user_id, symbol), group in grouped_data:
        daily_returns = group['profit_loss_percentage']
        mean_daily_return = daily_returns.mean()
        std_dev_daily_return = daily_returns.std()
        # Check if std_dev_daily_return is zero
        if std_dev_daily_return == 0:
            sharpe_ratio = np.nan  # Set to NaN or any other default value
        else:
            sharpe_ratio = (mean_daily_return / std_dev_daily_return) * np.sqrt(252)  # Assuming 252 trading days in a year
        result.append({'user_id': user_id, 'symbol': symbol, 'sharpe_ratio': sharpe_ratio})
    return pd.DataFrame(result)

sharpe_ratios = calculate_sharpe_ratio(df)
print(sharpe_ratios)

     user_id  symbol  sharpe_ratio
0       2056  ACLBSL    -12.615780
1       2056    ADLB           NaN
2       2056    AHPC     11.402528
3       2056     AIL           NaN
4       2056   AKBSL           NaN
..       ...     ...           ...
446    33387    SSHL    -35.139721
447    33387     STC     -8.017837
448    33387    SWMF           NaN
449    33387     TRH    -11.263348
450    33387   UNHPL           NaN

[451 rows x 3 columns]


## Profitable Trades

In [8]:
#percentage_profitable_trades = (df['profit_loss'] > 0).mean() * 100
def profitable_trades(df):
    # Group transactions by 'symbol' and 'user_id' and calculate the percentage of profitable trades for each combination
    profitable_trades_by_symbol_user = df.groupby(['user_id','symbol'])['profit_loss'].apply(lambda x: (x > 0).mean() * 100).reset_index()
    # Rename the columns for clarity
    profitable_trades_by_symbol_user.columns = ['user_id','symbol', 'Percentage_Profitable_Trades']
    # Print the result
    return profitable_trades_by_symbol_user
profitable_trade = profitable_trades(df)
print(profitable_trade)

     user_id  symbol  Percentage_Profitable_Trades
0       2056  ACLBSL                      7.142857
1       2056    ADLB                     50.000000
2       2056    AHPC                     42.857143
3       2056     AIL                     50.000000
4       2056   AKBSL                      0.000000
..       ...     ...                           ...
446    33387    SSHL                      0.000000
447    33387     STC                     25.000000
448    33387    SWMF                      0.000000
449    33387     TRH                     12.500000
450    33387   UNHPL                      0.000000

[451 rows x 3 columns]


In [9]:
dfs = [average_profit,average_cumulative_profit,average_profit,average_cumulative_profit,average_success_rate, sharpe_ratios,profitable_trade]
#dfs = [average_days_by_user_symbol,average_profit,average_cumulative_profit,average_profit,average_cumulative_profit,average_success_rate, sharpe_ratios,profitable_trade]

merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on=['user_id', 'symbol'], how='inner')
print(merged_df)

     user_id  symbol  avg_profit_in_each_trade_x  average_cumulative_profit_x  \
0       2056  ACLBSL                     -0.0400                    -0.128571   
1       2056    ADLB                      0.1800                     0.180000   
2       2056    AHPC                      0.0975                     0.112500   
3       2056     AIL                      0.0600                     0.060000   
4       2056   AKBSL                         NaN                          NaN   
..       ...     ...                         ...                          ...   
446    33387    SSHL                     -0.0700                    -0.162500   
447    33387     STC                     -0.0250                    -0.020000   
448    33387    SWMF                     -0.0200                    -0.020000   
449    33387     TRH                     -0.0350                    -0.135000   
450    33387   UNHPL                     -0.0600                    -0.060000   

     avg_profit_in_each_tra